In [1]:
from dotenv import load_dotenv
load_dotenv()
from langgraph.graph import StateGraph,END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [2]:
from uuid import uuid4

def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [3]:
tool = TavilySearchResults(max_results=2)


class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [4]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [5]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='a26ae330-800d-4321-bfae-b60e87958273'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1da2dabd-0f31-4111-8009-d92805a39c02-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 

In [6]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='a26ae330-800d-4321-bfae-b60e87958273'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1da2dabd-0f31-4111-8009-d92805a39c02-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'type': 'tool_call'}], usage_metadata={

In [7]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1738403406, \'localtime\': \'2025-02-01 01:50\'}, \'current\': {\'last_updated_epoch\': 1738403100, \'last_updated\': \'2025-02-01 01:45\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 0, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/296.png\', \'code\': 1183}, \'wind_mph\': 6.3, \'wind_kph\': 10.1, \'wind_degree\': 143, \'wind_dir\': \'SE\', \'pressure_mb\': 1023.0, \'pressure_in\': 30.2, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 90, \'cloud\': 100, \'fe

In [8]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='a26ae330-800d-4321-bfae-b60e87958273'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1da2dabd-0f31-4111-8009-d92805a39c02-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_p3byAUnHQMBxSROi0ODuLBPP', 'type': 'tool_call'}], usage_metadata={

In [9]:
abot.graph.get_state(thread).next

()

In [10]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='826f5a7d-1985-46c5-ac3a-fc1292b02347'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OZuVgW7p135kmBpEzBY1jEeY', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-01ba5737-5a64-4d7d-8593-92ea8f33233b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_OZuVgW7p135kmBpEzBY1jEeY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'out

### **Modifying**

In [11]:
messages = [HumanMessage("What is the weather in LA")]
thread = {"configurable": {"thread_id": 3}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
        print("-----")

{'messages': [HumanMessage(content='What is the weather in LA', id='5b0ab39a-95da-4ee3-976b-484112656742'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}], usage_metadata={'input_t

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA', id='5b0ab39a-95da-4ee3-976b-484112656742'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}], usa

In [13]:
current_values = abot.graph.get_state(thread)

In [14]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 23, 'total_tokens': 175})

In [15]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF',
  'type': 'tool_call'}]

In [16]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [17]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='What is the weather in LA', id='5b0ab39a-95da-4ee3-976b-484112656742'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF'}], usage_metadata={'input_tokens': 152, 'output_to

{'configurable': {'thread_id': '3',
  'thread_ts': '1efe0824-284d-65bc-8002-ad0904c3e284'}}

In [18]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA', id='5b0ab39a-95da-4ee3-976b-484112656742'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}], usage

In [19]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v),
        print("-----")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.4411, \'lon\': -91.0551, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1738403566, \'localtime\': \'2025-02-01 03:52\'}, \'current\': {\'last_updated_epoch\': 1738403100, \'last_updated\': \'2025-02-01 03:45\', \'temp_c\': -0.3, \'temp_f\': 31.5, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 6.7, \'wind_kph\': 10.8, \'wind_degree\': 54, \'wind_dir\': \'NE\', \'pressure_mb\': 1024.0, \'pressure_in\': 30.23, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 95, \'cloud\': 0, \'feelslike_c

### **Time Travel**

In [20]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA', id='5b0ab39a-95da-4ee3-976b-484112656742'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22638856-15e9-482d-9bb4-439bc0ebcc1f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_VBUEQhlapeGe8aIAkDZMTQOF', 'type': 'tool_call'}], usage

In [21]:
to_replay = states[-1]

In [22]:
to_replay

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '3', 'thread_ts': '1efe0822-f7ff-61dc-bfff-c3e2996a4ab6'}}, metadata={'source': 'input', 'step': -1, 'writes': {'messages': [HumanMessage(content='What is the weather in LA')]}}, created_at='2025-02-01T09:52:01.905916+00:00', parent_config=None)

In [23]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='What is the weather in LA', id='60bf069a-7f3e-496c-9908-a50092ab3e24'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6d6LZQV2Or0onFtdxXv1yxsj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-67358682-bb08-4bae-8ff3-a49bfbd09418-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_6d6LZQV2Or0onFtdxXv1yxsj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'ou